In [ ]:
import os
import sys
from dotenv import load_dotenv

load_dotenv(override=True)

paths = [
    os.path.abspath(os.path.join(os.getcwd(), "..")),
    os.path.abspath(os.path.join(os.getcwd(), "..", "agent")),
]

for dir_path in paths:
    if dir_path not in sys.path:
        sys.path.insert(0, dir_path)


In [ ]:
# get results texts
import os
results_dir = os.listdir("./results")
results = []
for result in results_dir:
    with open(f"./results/{result}", "r") as f:
        results.append(f.read())

In [ ]:
# extract Top 10 Investment Opportunities from the results
from pydantic import BaseModel, Field

from agent.llms import reasoning_model
from langchain_core.messages import HumanMessage, SystemMessage


class Item(BaseModel):
    suburb: str = Field(description="The suburb of the item.")
    address: str = Field(description="The address of the item.")
    rank: int = Field(description="The rank of the item.")


class ItemList(BaseModel):
    items: list[Item] = Field(
        description="The top 10 investment opportunities clearly listed in the report. If there are less than 10, just fill as many as you can."
    )


top_10_list: list[ItemList] = []

for result in results:
    messages = [
        SystemMessage(
            content="""
Read the report and extract the top 10 investment opportunities with the schema provided. Take an extra attention to not miss anything or include that is not in the report.
""".strip()
        ),
        HumanMessage(
            content=f"""
Here is the report:

{result}
""".strip()
        ),
    ]

    response: ItemList = reasoning_model.with_structured_output(ItemList).invoke(
        messages
    )
    top_10_list.append(response)


In [ ]:
# normalize the results
normalized_results = []